# 1. feladat [8p]

 Alex és Bob és Charlie közös programot szerveznek. A szabad időintervallumaik az intervals szótárban vannak megadva. Készítsünk programot, amely kiszámítja, hogy hány olyan nap van, amikor mindhárman ráérnekl Az intervallumok zártak, azaz a végpontok szabad időpontnak tekintendők. A program ne csak a megadott intervals adatszerkezetre működjön, hanem tetszőleges, ugyanilyen formátumú bemenetre is! 

In [ ]:
import datetime as dt

intervals = {
    'Alex': [('2020-06-20','2020-06-23'), ('2020-06-30', '2020-07-05')],
    'Bob' : [('2020-06-18','2020-06-21'), ('2020-06-24','2020-07-01'), ('2020-07-03', '2020-07-04')],
    'Charlie': [('2020-06-21', '2020-06-28'), ('2020-07-02', '2020-07-10')],
}

def get_free_dates(name, intervals):
    free_dates = []
  
    free_periods = intervals[name]

    for period in free_periods:
       start_date = dt.datetime.strptime(period[0], "%Y-%m-%d").date()
       end_date = dt.datetime.strptime(period[1], "%Y-%m-%d").date()

       free_dates_period = [start_date + dt.timedelta(days=current_day) for current_day in range((end_date - start_date).days + 1)]
       free_dates.extend([date for date in free_dates_period])

    return free_dates

def main():
    names = intervals.keys()

    req_occurs = len(names)
    
    free_dates_nested = [get_free_dates(name, intervals) for name in names]
    free_dates = [date for free_date_for_name in free_dates_nested for date in free_date_for_name]
    
    correct_dates = {date for date in free_dates if free_dates.count(date) == req_occurs}
    print(len(correct_dates))
    
main()

# 2. feladat [12p] 

A china.txt Kína mezőgazdaságáról tartalmaz összesitő adatokat, az 1949 és 2008 közötti időszakból. Készitsünk programot, amely beolvassa a szövegfájl tartalmát, majd válaszol az alábbi kérdésekre! 
- Összesen hány tonna trágyát használtak a 60-as években? (Áru: Total fertilizer consumption ) 
- Melyik évben volt a legmagasabb a mezőgazdaságban dolgozók száma? (Áru: Ag employment (primary industry) ) 
- A rizstermeles (Kategória: Crop production , Áru: Rice ) melyik évben hányszorosa volt az 1949-es értéknek? 
- 2007-ben melyik 5 terménynek volt átlagosan a legnagyobb a hektáronkénti hozama? (A termelési mennyiségek a crop production , a vetési területek a crop sown area kategóriában van megadva.) 


In [ ]:
# IDE JÖN A MEGOLDÁS!